# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,7.11,91,100,2.40,SJ,1719312636
1,1,ulladulla,-35.3500,150.4667,10.83,71,7,2.42,AU,1719312747
2,2,port-aux-francais,-49.3500,70.2167,4.27,98,100,6.70,TF,1719312749
3,3,emerald,-23.5333,148.1667,19.08,64,92,5.64,AU,1719312750
4,4,qaqortoq,60.7167,-46.0333,2.92,87,94,1.63,GL,1719312752


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    size='Humidity', 
    color = 'City',
    hover_cols=['City', 'Humidity'],
    geo=True,
    tiles='OSM',
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 20) & 
    (city_data_df["Max Temp"] < 26) & 
    (city_data_df["Humidity"] < 50) & 
    (city_data_df["Cloudiness"] < 75)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,kedrovyy,56.1667,91.8167,25.36,44,5,3.84,RU,1719312930
124,124,tsarychanka,48.9370,34.4786,24.40,44,60,6.62,UA,1719312942
133,133,kargopol',61.5071,38.9456,21.97,44,14,2.76,RU,1719312957
198,198,plesetsk,62.7080,40.2916,21.22,37,4,3.04,RU,1719313062
210,210,kabare,-2.4683,28.8242,23.92,46,8,1.71,CD,1719313080
221,221,nephi,39.7102,-111.8363,21.52,43,35,1.93,US,1719312878
270,270,altai,46.3722,96.2583,23.28,25,43,4.52,MN,1719313179
280,280,palmas,-10.2128,-48.3603,24.74,43,20,1.74,BR,1719313195
308,308,shakawe,-18.3667,21.8500,24.21,30,0,3.97,BW,1719313240
372,372,tevriz,57.5090,72.4042,25.67,47,8,5.64,RU,1719313342


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
116,kedrovyy,RU,56.1667,91.8167,44,
124,tsarychanka,UA,48.9370,34.4786,44,
133,kargopol',RU,61.5071,38.9456,44,
198,plesetsk,RU,62.7080,40.2916,37,
210,kabare,CD,-2.4683,28.8242,46,
221,nephi,US,39.7102,-111.8363,43,
270,altai,MN,46.3722,96.2583,25,
280,palmas,BR,-10.2128,-48.3603,43,
308,shakawe,BW,-18.3667,21.8500,30,
372,tevriz,RU,57.5090,72.4042,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kedrovyy - nearest hotel: No hotel found
tsarychanka - nearest hotel: Оріль
kargopol' - nearest hotel: Каргополь
plesetsk - nearest hotel: Плесецкая
kabare - nearest hotel: Panorama Hotel
nephi - nearest hotel: Best Western Paradise Inn
altai - nearest hotel: Altai hotel 40000-100000T
palmas - nearest hotel: Hotel Triângulo Mineiro
shakawe - nearest hotel: EdenWood guesthouse
tevriz - nearest hotel: No hotel found
eagle mountain - nearest hotel: No hotel found
jales - nearest hotel: No hotel found
hanak - nearest hotel: No hotel found
beitbridge - nearest hotel: Beitbridge Inn & Casino
moyale - nearest hotel: Paul’s guest house
kununurra - nearest hotel: No hotel found
mwinilunga - nearest hotel: No hotel found
ostersund - nearest hotel: Hotell Stortorget
nesoddtangen - nearest hotel: Scandic Fornebu
mende - nearest hotel: O'Mineola
ga-kgapane - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
mahenge - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
116,kedrovyy,RU,56.1667,91.8167,44,No hotel found
124,tsarychanka,UA,48.9370,34.4786,44,Оріль
133,kargopol',RU,61.5071,38.9456,44,Каргополь
198,plesetsk,RU,62.7080,40.2916,37,Плесецкая
210,kabare,CD,-2.4683,28.8242,46,Panorama Hotel
221,nephi,US,39.7102,-111.8363,43,Best Western Paradise Inn
270,altai,MN,46.3722,96.2583,25,Altai hotel 40000-100000T
280,palmas,BR,-10.2128,-48.3603,43,Hotel Triângulo Mineiro
308,shakawe,BW,-18.3667,21.8500,30,EdenWood guesthouse
372,tevriz,RU,57.5090,72.4042,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    size='Humidity', 
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    geo=True,
    tiles='OSM',
    title='City Humidity and Hotel Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)